# Web Scraping of state polices departments from www.50states.com

In [222]:
import requests
from lxml import html
import pandas as pd
from requests.structures import CaseInsensitiveDict

### Define List of States

In [223]:

states_names ={
               "Alabama":"AL","Alaska":"AK",
               "Arizona":"AZ",
               "Arkansas":"AR","California":"CA","Colorado":"CO",
               "Connecticut":"CT","Delaware":"DE","Florida":"FL","Georgia":"GA","Hawaii":"HI",
               "Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS","Kentucky":"KY",
               "Louisiana":"LA","Maine":"ME",
               "Maryland":"MD","Massachusetts":"MA","Michigan":"MI","Minnesota":"MN","Mississippi":"MS",
               "Missouri":"MO","Montana":"MT",
               "Nebraska":"NE","Nevada":"NV","New_Hampshire":"NH","New_Jersey":"NJ","New_Mexico":"NM",
               "New_York":"NY","North_Carolina":"NC",
               "North_Dakota":"ND","Oregon":"OR","Pennsylvania":"PA","Rhode_Island":"RI",
               "South_Carolina":"SC","South_Dakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT","Vermont":"VT",
               "Virginia":"VA","Washington":"WA","West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY"
               }

### Define list variables


In [249]:
name_list = []
address_list = []
state_list = []

## Define functions

### get_50states_static_page
 - Get 50states.com html page content.
 - Input - None
 - Output - Dictionary 
     - state_name - list of states names
     - states_list - list of content of html page
     - elements_list - count of elements to extract(acount of all police departments per state)

In [250]:
def get_50states_static_page (statesNames):

    states_dict = dict()
    states_list = []
    elements_list = []
    
    for state_name,state_abbr in statesNames.items():
        
        page = requests.get('https://www.50states.com/'+state_name+'/police_departments.htm')
        tree = html.fromstring(page.content)
        count_elements = tree.xpath('count(//strong)')
        
        #print(state_name+" "+str(count_elements))
        states_list.append(tree)
        elements_list.append(int(count_elements))
        states_dict[state_abbr]=state_name,states_list,elements_list
        #print(type(states_dict))
        
    return states_dict

### get_names_addresses
 - extract names and addresses from html object
 - Input - Dictionary
 - Output - lists 
     - name_list - list of states names
     - address_list - list of departments addresses
     - state_list - list of state abbreviations

In [287]:
def get_names_addresses(states):
    
    i=0
    for key, value in states.items():
        try:
    
            #print(key)
            elements = value[2][i]
            html = value[1][i]
            #print(elements)
            i=i+1

            for x in  range(1,elements):
                #print(x)
                depName = str(html.xpath('/html/body/div[3]/div[3]/div/div[1]/div[2]/main/div/article/div/ul/li['+str(x)+']/strong/text()')).replace('[\'', '').replace('\']', '').replace('\"', '').replace('[', '').replace(']', '')
                address = html.xpath('/html/body/div[3]/div[3]/div/div[1]/div[2]/main/div/article/div/ul/li['+str(x)+']/text()')
                if len(depName)>3:
                    name_list.append(depName)
                    address_list.append(address[3]+','+address[4])
                    state_list.append(state_abbr)
        except IndexError:
            pass
    return name_list,address_list,state_list


## Add data to daframe

In [289]:
names,address,states = get_names_addresses(get_50states_static_page (states_names))


In [290]:
df = pd.DataFrame(columns = ['name','address','state'])

df["name"] = names
df["address"] = address
df["state"] = states

df.head(6)

,name,address,state
0,Abbeville Police Dept,"101 E Washington St, Abbeville, 36310",WY
1,Albertville City Police Dept,"116 W Main St, Albertville, 35950",WY
2,Aliceville Police Dept,"215 1st St NE, Aliceville, 35442",WY
3,Altoona Police Dept,"2844 Main St, Altoona, 35952",WY
4,Animal Control,"934 N Ripley St, Montgomery, 36104",WY
5,Arab Police Dept,"1447 Hog Jaw Rd, Arab, 35016",WY


## Dataframe to dictionary

In [302]:
df2dict = df.to_dict("index")

### Use geoapify to load 

In [305]:
def get_coordinates(addresses):
    for keys,values  in addresses.items():
        try:

            #print(df2dict[keys]["address"])

            url2api=addresses[keys]["address"].replace(",","").replace(" ","%20").replace(".","")
            #print(url2api)

            url = "https://api.geoapify.com/v1/geocode/search?text="+url2api+"&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6"
            #url = "https://api.geoapify.com/v1/geocode/search?text=Post%20Office%20Box%20308%20Abbeville%20AL%2036310&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6"
            print(url)
            headers = CaseInsensitiveDict()
            headers["Accept"] = "application/json"
            #print(resp.json())

            resp = requests.get(url, headers=headers)
            print(resp.status_code)
            addresses[keys]["longitude"] = resp.json()["results"][0]["lon"]
            addresses[keys]["latitude"] = resp.json()["results"][0]["lat"]

            #print(resp.json()["results"][0]["lat"])
            
        except Exception as e:
            print(e)
            addresses[keys]["longitude"] = [0,0]
            addresses[keys]["latitude"] = [0,0]

    return addresses                   

In [306]:
data = get_coordinates(df2dict)

https://api.geoapify.com/v1/geocode/search?text=101%20E%20Washington%20St%20Abbeville%2036310&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=116%20W%20Main%20St%20Albertville%2035950&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=215%201st%20St%20NE%20Aliceville%2035442&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=2844%20Main%20St%20Altoona%2035952&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=934%20N%20Ripley%20St%20Montgomery%2036104&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1447%20Hog%20Jaw%20Rd%20Arab%2035016&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=740%20N%20Main%20St%20Arab%2035016&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017

200
https://api.geoapify.com/v1/geocode/search?text=14521%20Al%20Highway%2068%20Crossville%2035962&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=730%20S%20Daleville%20Ave%20Daleville%2036322&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1502%20Highway%2098%20Daphne%2036526&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1502%20US%20Highway%2098%20Daphne%2036526&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1502%20Highway%2098%20Daphne%2036526&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1502%20US%20Highway%2098%20Daphne%2036526&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1015%20Desoto%20Ave%20Dauphin%20Island%2036528&format=json&apiKey=2fc3d62

200
https://api.geoapify.com/v1/geocode/search?text=11355%20Main%20St%20Hillsboro%2035643&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=610%20Martin%20Luther%20King%20Dr%20Anniston%2036201&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=29164%20US%20Hwy%2072%20Hollywood%2035752&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1833%2029th%20Ave%20S%20Homewood%2035209&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1833%2029th%20Ave%20S%20Homewood%2035209&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1833%2029th%20Ave%20S%20Homewood%2035209&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=100%20Municipal%20Dr%20Birmingham%2035216&format=json&apiKey=2fc3d62

200
https://api.geoapify.com/v1/geocode/search?text=5496%20Main%20Dr%20New%20Hope%2035760&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=100%20King%20St%20Newton%2036352&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1910%20Lurleen%20B%20Wallace%20Blvd%20Northport%2035476&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=76%20W%20Main%20St%20Notasulga%2036866&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=10294%20Main%20St%20Oakman%2035579&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=111%20Jack%20Fendley%20Dr%20Oneonta%2035121&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=111%20Jack%20Fendley%20Dr%20Oneonta%2035121&format=json&apiKey=2fc3d62d73f04156

200
https://api.geoapify.com/v1/geocode/search?text=1300%20Alabama%20Ave%20Selma%2036703&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1300%20Alabama%20Ave%20Selma%2036703&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1300%20Alabama%20Ave%20Selma%2036703&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1300%20Alabama%20Ave%20Selma%2036703&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=370%201st%20St%20Silas%2036919&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=22030%20Sixth%20St%20Silverhill%2036576&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=255%20E%20Lawrence%20Harris%20Hwy%20Slocomb%2036375&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
2

...

200
https://api.geoapify.com/v1/geocode/search?text=301%20Mill%20St%20Orangeville%2017859&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=209%20N%20Warren%20St%20Orwigsburg%2017961&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=Boro%20Bldg%20Osceola%20Mills%2016666&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1538%20Commerce%20Ave%20Carlisle%2017013&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=350%20Scarlet%20Rd%20Kennett%20Square%2019348&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=5730%20Interchange%20Rd%20Lehighton%2018235&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=5%20Weller%20Pl%20Easton%2018045&format=json&apiKey=2fc3d62d73f04156b0937e

200
https://api.geoapify.com/v1/geocode/search?text=6666%20Ridge%20Ave%20Philadelphia%2019128&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=1701%20Bowler%20St%20Philadelphia%2019115&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=3100%20Red%20Lion%20Rd%20Philadelphia%2019114&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=3800%20Lincoln%20Dr%20Philadelphia%2019144&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=401%20N%2021st%20St%20Philadelphia%2019130&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=8501%20State%20Rd%20Philadelphia%2019136&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=2301%20S%2024th%20St%20Philadelphia%2019145&format=json&apiKey=2fc3

200
https://api.geoapify.com/v1/geocode/search?text=123%20Walnut%20St%20Harrisburg%2017101&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=123%20Walnut%20St%20Harrisburg%2017101&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=815%20Washington%20St%20Reading%2019601&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=815%20Washington%20St%20Reading%2019601&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=660%201st%20Ave%20Pittsburgh%2015219&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=123%20Walnut%20St%20#215a%20Harrisburg%2017101&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
401
'results'
https://api.geoapify.com/v1/geocode/search?text=10%20N%202nd%20St%20#103%20Harrisburg%2017101&format=json&apiKey=2fc3d62

200
https://api.geoapify.com/v1/geocode/search?text=300%20Saint%20Catherine%20St%20Quarryville%2017566&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=Rr%201%20Aliquippa%2015001&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=301%20Iven%20Ave%20Radnor%2019087&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=28%20S%20Market%20St%20Elysburg%2017824&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=815%20Washington%20St%20Reading%2019601&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=11%20E%20Broadway%20Red%20Lion%2017356&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https://api.geoapify.com/v1/geocode/search?text=128%205th%20St%20Renovo%2017764&format=json&apiKey=2fc3d62d73f04156b0937e52aa7017f6
200
https

KeyboardInterrupt: 

In [307]:
data

NameError: name 'data' is not defined

In [297]:
v

NameError: name 'v' is not defined

In [ ]:
df_lon_lat = pd.DataFrame.from_dict(data, orient='index',  columns=['name','address','state','coordinates'])

In [ ]:
path = r"C:\Users\Alina\NIBRS_target\facts"
import os
df_lon_lat.to_csv(os.path.join(path,"agencies_lon_lat_corr.csv"),index=True) 